In [9]:
import numpy as np
import pandas as pd

In [10]:
bar = 20 #45
#hw = 5*[5] + 2*[2.5] + 2*[1.25] 
hw = [44] + [33] + [22] + [11] + 2*[5.5] + 2*[2.75]
hw

[44, 33, 22, 11, 5.5, 5.5, 2.75, 2.75]

In [11]:
2 * sum(hw)

253.0

In [25]:
n_comb = 2**len(hw)
n_comb

256

In [27]:
data = [
    [int(i) for i in list(f'{str(bin(n))[2:]:>08s}')]   # 08s = len(hw)
    for n in range(n_comb)]
data[:3]

[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0]]

In [28]:
df = pd.DataFrame(np.array(data) * np.array(hw), columns=hw)
df.head()

,44.00,33.00,22.00,11.00,5.50,5.50,2.75,2.75
0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2.75
2,0.0,0.0,0.0,0.0,0.0,0.0,2.75,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,2.75,2.75
4,0.0,0.0,0.0,0.0,0.0,5.5,0.00,0.00


In [29]:
df['halfweight'] = df.sum(axis=1)
df['weight'] = df.halfweight * 2
df['total'] = df.weight + bar
df.drop_duplicates(subset=['total'], inplace=True)
df = df.sort_values('total')
df.head(20)

,44.0,33.0,22.0,11.0,5.5,5.5,2.75,2.75,halfweight,weight,total
0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,20.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2.75,2.75,5.5,25.5
3,0.0,0.0,0.0,0.0,0.0,0.0,2.75,2.75,5.50,11.0,31.0
5,0.0,0.0,0.0,0.0,0.0,5.5,0.00,2.75,8.25,16.5,36.5
7,0.0,0.0,0.0,0.0,0.0,5.5,2.75,2.75,11.00,22.0,42.0
13,0.0,0.0,0.0,0.0,5.5,5.5,0.00,2.75,13.75,27.5,47.5
15,0.0,0.0,0.0,0.0,5.5,5.5,2.75,2.75,16.50,33.0,53.0
21,0.0,0.0,0.0,11.0,0.0,5.5,0.00,2.75,19.25,38.5,58.5
23,0.0,0.0,0.0,11.0,0.0,5.5,2.75,2.75,22.00,44.0,64.0
29,0.0,0.0,0.0,11.0,5.5,5.5,0.00,2.75,24.75,49.5,69.5


In [30]:
df.to_csv('~/Desktop/strength/weights.csv', index=False)

In [5]:
c = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMONPQRSTUVWKYZ0123456789$.,!%^*#')
''.join(np.random.choice(c, size=15))

'%MY26scYio%1rpw'

In [2]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [3]:
def separate_accounts(fid):
    accounts = {
        account_name: fid[fid['Account Name'] == account_name]
        for account_name in fid['Account Name'].unique()}
    for acct in accounts:
        accounts[acct].index = accounts[acct].Symbol
        accounts[acct] = (
            accounts[acct]
            .rename(columns={'Current Value': acct})
            .drop(columns=['Account Name', 'Symbol']))
    return accounts

In [8]:
def upload_fidelity_deprecated():
    print('Uploading Fidelity data...')
    fidelity = pd.read_csv(f'~/Desktop/Portfolio_Positions_May-03-2024.csv')
    fidelity = (
        fidelity[['Account Name', 'Symbol', 'Current Value']].dropna())
    fidelity['Current Value'] = (
        fidelity['Current Value']
        .apply(convert_value)
        .fillna(0)
        .astype(int))
    fidelity = fidelity[
        ((fidelity.Symbol != 'SPAXX**')
         & (fidelity.Symbol != 'Pending Activity'))]
    fidelity = separate_accounts(fidelity)
    fidelity = pd.concat([df for df in fidelity.values()], axis=1)
    fidelity.columns = ['rollover', 'roth', 'simple']
    fidelity['fid'] = fidelity.sum(axis=1)
    return fidelity

In [9]:
f1 = upload_fidelity_deprecated()
f1.head()

Uploading Fidelity data...


,rollover,roth,simple,fid
Symbol,,,,
NET,446.0,149.0,NaN,595.0
SBUX,2120.0,2705.0,NaN,4825.0
MCD,2974.0,4055.0,NaN,7029.0
HDB,4388.0,293.0,NaN,4681.0
JNJ,5374.0,NaN,NaN,5374.0


In [10]:
print('Uploading Fidelity data...')
filename = 'Accounts_History.csv'
print('Looking for Fidelity file:', filename)
fidelity = pd.read_csv(f'~/Downloads/{filename}')
fidelity = (
    fidelity[['Account', 'Symbol', 'Current Value']].dropna())

Uploading Fidelity data...
Looking for Fidelity file: Accounts_History.csv


,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
0,05/14/2024,ROTH IRA 221247008,YOU BOUGHT MASIMO CORP (MASI) (Cash),MASI,MASIMO CORP,Cash,1.0,115.85,NaN,NaN,NaN,-115.85,05/16/2024
1,05/13/2024,ROTH IRA 221247008,YOU SOLD UPSTART HLDGS INC COM (UPST) (Cash),UPST,UPSTART HLDGS INC COM,Cash,-16.0,27.65,NaN,NaN,NaN,442.40,05/15/2024
2,05/13/2024,ROTH IRA 221247008,YOU SOLD RIVIAN AUTOMOTIVE INC COM CL A (RIVN...,RIVN,RIVIAN AUTOMOTIVE INC COM CL A,Cash,-524.0,10.60,NaN,0.05,NaN,5554.35,05/15/2024
3,05/13/2024,ROTH IRA 221247008,YOU SOLD REVOLVE GROUP INC CL A (RVLV) (Cash),RVLV,REVOLVE GROUP INC CL A,Cash,-8.0,22.28,NaN,NaN,NaN,178.24,05/15/2024
4,05/13/2024,ROTH IRA 221247008,YOU SOLD JD.COM INC SPON ADS EACH REPR 2 ORD ...,JD,JD.COM INC SPON ADS EACH REPR 2 ORD SHS,Cash,-3.0,33.41,NaN,NaN,NaN,100.23,05/15/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,purposes. For more information on the data inc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Brokerage services are provided by Fidelity Br...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Financial Services LLC (NFS). Both are Fidelit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,"Fidelity Insurance Agency, Inc., and, for cert...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
